In [79]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import liana as li
import anndata
import scanpy as sc
import os 
import scipy

In [80]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from utils import *

2.9.1+cpu


In [81]:
import argparse
import os.path as osp

import torch
import torch.nn.functional as F

from torch_geometric.datasets import Entities
from torch_geometric.nn import FastRGCNConv, RGCNConv, GCNConv, InnerProductDecoder, GAE, VGAE
from torch_geometric.utils import k_hop_subgraph

from torch_geometric.datasets import Planetoid
from torch_geometric.data.data import Data
from torch_geometric.transforms import NormalizeFeatures


from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import SpectralClustering
import random
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity

from scipy.io import mmread

from scipy.stats import wilcoxon
import anndata
import scanpy as sc

In [82]:
pwd

'c:\\Users\\chowz\\OneDrive\\umich\\2025 Fall\\BIOINF593\\final_project\\remote_repo\\GraphComm\\code\\preprocessing'

## Processing with csv file

In [83]:
matrix = pd.read_csv("../../data/raw_data/Drosophila/matrix.csv",index_col=0)

In [84]:
matrix

,GTACTAATTACN_2,AGGCTAATGGAC_3,TAGACAAAAGCT_4,TCTACTAGTGTN_5,CCGTGCTGAACA_3,GAGAAAATGAAG_7,GCCTAGTGACGT_4,CAGTATCGAGAN_2,AGATTACCCTTC_6,AACCCCACGCTC_2,...,TAGAGGACCGTA_3,TTGGGTAACAGC_3,GTGAGCTTATGG_3,GCTTCTCCGTCT_3,GCTTGGAAATCC_7,GCGATCAGTCTT_3,AGAACGATATAC_8,CCGATTCGCTTN_4,CCTCTCAGTCTC_5,GACTCTCTCGCA_4
128UP,2.570,2.509,0.000,0.000,0.000,0.000,0.000,2.556,2.507,2.343,...,0.000,0.000,3.526,0.000,0.000,0.000,0.000,1.972,0.000,0.000
140UP,1.160,3.007,0.000,0.000,0.000,3.249,3.208,3.967,0.000,0.000,...,3.470,0.000,3.526,3.125,2.834,3.272,3.411,2.775,0.000,0.000
14-3-3EPSILON,8.838,8.788,8.843,8.869,8.345,8.999,8.788,8.860,8.624,8.730,...,8.383,8.186,9.099,8.553,8.809,8.898,8.029,8.333,8.554,7.780
14-3-3ZETA,5.942,5.434,6.017,5.016,6.010,5.815,5.657,5.324,6.635,6.048,...,4.404,5.459,6.992,5.881,6.334,6.588,5.620,5.172,4.640,5.318
18W,3.443,3.007,2.318,4.060,3.533,2.394,3.208,2.556,0.000,0.000,...,3.470,0.000,0.000,0.000,3.729,0.000,0.000,0.000,5.611,3.423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZW10,2.842,3.007,2.318,2.299,0.000,3.249,2.356,2.556,2.507,0.000,...,0.000,0.000,0.000,3.654,0.000,0.000,0.000,4.212,3.166,0.000
ZWILCH,0.000,1.742,2.318,3.144,0.000,0.000,0.000,0.000,0.000,3.725,...,4.404,0.000,0.000,3.125,0.000,0.000,0.000,0.000,0.000,0.000
ZYD,1.160,1.742,0.000,2.299,3.009,0.000,2.356,2.556,0.000,0.000,...,0.000,4.492,0.000,2.281,0.000,0.000,3.411,0.000,0.000,0.000
ZYE,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [85]:
adata = anndata.AnnData(matrix.transpose())

In [86]:
adata

AnnData object with n_obs × n_vars = 1297 × 8924

In [87]:
sc.pp.filter_genes(adata, min_cells=5)
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.neighbors(adata)
sc.tl.leiden(adata)

In [88]:
meta = pd.DataFrame({"cell":adata.obs["leiden"].index.tolist(),"labels":adata.obs["leiden"].tolist()})

In [89]:
meta.index = meta["cell"].tolist()

In [90]:
adata.obs = meta

In [91]:
matrix = pd.DataFrame(adata.X.transpose(),columns=adata.obs.index.tolist(),index=adata.var.index.tolist())

# Processing with h5ad

In [ ]:

# adata = sc.read_h5ad("../../data/raw_data/Cardiac_cells/Visium-RZ_FZ_P5.h5ad")
# matrix = pd.DataFrame.sparse.from_spmatrix(adata.X,index=adata.obs.index.tolist(),columns=adata.var["feature_name"].tolist())
# sc.pp.filter_genes(adata, min_cells=10)
# sc.pp.normalize_total(adata)
# sc.pp.log1p(adata)
# sc.pp.neighbors(adata)
# sc.tl.leiden(adata)
# matrix = pd.DataFrame.sparse.from_spmatrix(adata.X,index=adata.obs.index.tolist(),columns=adata.var["feature_name"].tolist())
# meta = pd.DataFrame({"cell":adata.obs.index.tolist(),"labels":adata.obs["cell_type_original"].tolist()})
# meta.index = meta["cell"].tolist()
# #removing lymphoid as it is only one cell, cannot be used in benchmarking
# meta = meta[meta['labels'] != "Lymphoid"]
# adata = adata[meta["cell"].tolist()]
# matrix = matrix.transpose()

# matrix = matrix[meta.index.tolist()]

# index = matrix.index.tolist()
# adata.obs = meta


# rest of preprocessing

In [93]:
#sc.tl.rank_genes_groups(adata, 'labels')
cell_groups = meta['labels'].unique().tolist()
matrix_list = {}
for i in cell_groups:
    cells = meta[meta["labels"]==i].index.tolist()
    temp_matrix = matrix[cells]
    matrix_list[i] = (temp_matrix.mean(axis=1)[temp_matrix.mean(axis=1) >= 0].index.tolist())

In [94]:
adata = adata.transpose()

In [95]:
cell_type_df = matrix_list

In [96]:
#mean_expression = np.mean(matrix.values[matrix.values != 0])
nodes = pd.DataFrame({"category":[],"identifier":[]})

In [97]:
LR_nodes = pd.read_csv("../../data/LR_database/intercell_nodes.csv",index_col=0)
Omnipath_network = pd.read_csv("../../data/LR_database/intercell_interactions.csv",index_col=0)

In [98]:
ligands = LR_nodes[LR_nodes["category"]=="Ligand"]["identifier"].tolist()
receptors = LR_nodes[LR_nodes["category"]=="Receptor"]["identifier"].tolist()

In [99]:
ligand_list = []
receptor_list = []
new_cell_df = {}
for i in cell_type_df.keys():
    ligand_list.extend(list(set(ligands) & set(cell_type_df[i])))
    receptor_list.extend(list(set(receptors) & set(cell_type_df[i])))
    new_cell_df[i] = [list(set(ligands) & set(cell_type_df[i])),list(set(receptors) & set(cell_type_df[i]))]

In [100]:
for i in new_cell_df.keys():
    new_cell_df[i][0] = [j+"_Ligand" for j in new_cell_df[i][0]]
    new_cell_df[i][1] = [j+"_Receptor" for j in new_cell_df[i][1]]

In [101]:
ligand_list = list(set(ligand_list))
receptor_list = list(set(receptor_list))

In [102]:
ligand_list = [i+"_Ligand" for i in ligand_list]
receptor_list = [i+"_Receptor" for i in receptor_list]

In [103]:
nodes = pd.concat([nodes,pd.DataFrame({"category":["Cell Group"] * len(list(cell_type_df.keys())), "identifier":list(cell_type_df.keys())})])
nodes = pd.concat([nodes,pd.DataFrame({"category":["Ligand"] * len(ligand_list), "identifier":ligand_list})])
nodes = pd.concat([nodes,pd.DataFrame({"category":["Receptor"] * len(receptor_list), "identifier":receptor_list})])

In [104]:
new_identifier = [row["identifier"] + "_" + row["category"] for index,row in LR_nodes.iterrows()]

In [105]:
LR_nodes["identifier"] = new_identifier

In [106]:
nodes["Id"] = range(0,nodes.shape[0])
nodes = nodes[["Id","category","identifier"]]

In [107]:
nodes.index = nodes.index.astype('int')
nodes["Id"] = nodes["Id"].astype('int')

In [108]:
meta.index = meta["cell"].tolist()

In [109]:
interactions = pd.DataFrame({"Src":[],"Dst":[],"Weight":[],"edge_type":[]})

In [110]:
LR_nodes.index = LR_nodes["Id"].tolist()

In [111]:
LR_nodes

,Id,category,identifier
1,1,Ligand,EPOR_Ligand
2,2,Ligand,FYN_Ligand
3,3,Ligand,KL_Ligand
4,4,Ligand,S100A10_Ligand
5,5,Ligand,JAG2_Ligand
...,...,...,...
1955,1955,Receptor,PRKCD_Receptor
1956,1956,Receptor,SLK_Receptor
1957,1957,Receptor,STK38_Receptor
1958,1958,Receptor,STK11_Receptor


In [112]:
Omnipath_network["Src"] += 1
Omnipath_network["Dst"] += 1

In [113]:
Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["identifier"].tolist()
Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["identifier"].tolist()


In [114]:
interactions

,Src,Dst,Weight,edge_type


In [115]:
source_list = []
dest_list = []
weight_list = []
edge_type_list = []
for i in new_cell_df.keys():
    source_list.extend([i] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))
    dest_list.extend(new_cell_df[i][0])
    dest_list.extend(new_cell_df[i][1])
    weight_list.extend([1] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))
    edge_type_list.extend([1] * (len(new_cell_df[i][0]) + len(new_cell_df[i][1])))

In [116]:
interactions["Src"] = source_list
interactions["Dst"] = dest_list
interactions["Weight"] = weight_list
interactions["edge_type"] = edge_type_list

In [117]:
interactions #this could be where we edit weight

,Src,Dst,Weight,edge_type
0,6,WNT2_Ligand,1,1
1,6,PIP_Ligand,1,1
2,6,FAS_Ligand,1,1
3,6,FST_Ligand,1,1
4,6,CORT_Ligand,1,1
...,...,...,...,...
355,0,CSK_Receptor,1,1
356,0,LRP1_Receptor,1,1
357,0,RACK1_Receptor,1,1
358,0,PVR_Receptor,1,1


In [118]:
nodes.index = nodes["identifier"].tolist()

In [119]:
nodes = nodes.drop_duplicates("identifier")
nodes["Id"] = range(0,nodes.shape[0])

In [120]:
interactions["Src"] = nodes.loc[interactions["Src"].tolist()]["Id"].tolist()
interactions["Dst"] = nodes.loc[interactions["Dst"].tolist()]["Id"].tolist()

In [121]:
first_data,first_nodes,first_interactions = make_dataset(nodes,interactions,first=False,pathway_encode=False)

# Edit the ground truth to only contain information from this dataset

In [122]:
LR_nodes.index = LR_nodes["identifier"].tolist()

In [123]:
#LR_nodes["identifier"] = [row["identifier"] + "_" + row["category"] for index,row in LR_nodes.iterrows()]
LR_nodes = LR_nodes[(LR_nodes["identifier"].isin(ligand_list)) | (LR_nodes["identifier"].isin(receptor_list))]
Omnipath_network = Omnipath_network[(Omnipath_network["Src"].isin(LR_nodes["identifier"].tolist())) & (Omnipath_network["Dst"].isin(LR_nodes["identifier"].tolist()))]

In [124]:
ligand_list = Omnipath_network["Src"].tolist()
receptor_list = Omnipath_network["Dst"].tolist()
LR_nodes = LR_nodes[(LR_nodes["identifier"].isin(ligand_list)) | (LR_nodes["identifier"].isin(receptor_list))]
Omnipath_network = Omnipath_network[(Omnipath_network["Src"].isin(LR_nodes["identifier"].tolist())) & (Omnipath_network["Dst"].isin(LR_nodes["identifier"].tolist()))]

In [125]:
ligand_list = Omnipath_network["Src"].tolist()
receptor_list = Omnipath_network["Dst"].tolist()

In [126]:
LR_nodes["Id"] = range(0,LR_nodes.shape[0])
LR_nodes.index = LR_nodes["identifier"].tolist()
Omnipath_network["Src"] = LR_nodes.loc[Omnipath_network["Src"].tolist()]["Id"].tolist()
Omnipath_network["Dst"] = LR_nodes.loc[Omnipath_network["Dst"].tolist()]["Id"].tolist()

In [127]:
interactions["edge_type"]=1

In [128]:
# os.system("mkdir -p ../../data/GraphComm_Input/intercell_Drosophila")

os.makedirs("../../data/GraphComm_Input/intercell_Drosophila", exist_ok=True)

In [129]:
nodes.to_csv("../../data/GraphComm_Input/intercell_Drosophila/nodes.csv")
interactions.to_csv("../../data/GraphComm_Input/intercell_Drosophila/interactions.csv")
meta.to_csv("../../data/GraphComm_Input/intercell_Drosophila/meta.csv")
matrix.to_csv("../../data/GraphComm_Input/intercell_Drosophila/matrix.csv")
LR_nodes.to_csv("../../data/GraphComm_Input/intercell_Drosophila/LR_nodes.csv")
Omnipath_network.to_csv("../../data/GraphComm_Input/intercell_Drosophila/Omnipath_network.csv")